In [ ]:
# Project Name: Chain of Thought Reasoning
# Team Number: 6
# Members: Anvesh, Eshita, Neha, Sandeep, Saumya

# File Name: airflow-llama-ft-eval.ipynb
# File Usage: 
# Used for calculating the g-eval score for the fine-tuned Llama model on the Chain of Thought Reasoning dataset.
#   - To evaluate the fine-tuned Llama model for Chain of Thought Reasoning
#   - To generate predictions using the fine-tuned model
#   - To calculate the accuracy of the predictions
#   - To save the predictions to a CSV file


# Metadata:
# {
#     "id": "eshitagupta151991/airflow-llama-ft-v2",
#     "title": "airflow-llama-ft-v2",
#     "code_file": "airflow-llama-ft.ipynb",
#     "language": "python",
#     "kernel_type": "notebook",
#     "is_private": true,
#     "enable_gpu": true,
#     "enable_tpu": false,
#     "enable_internet": true,
#     "dataset_sources": ["eshitagupta151991/secrets"],
#     "competition_sources": [],
#     "kernel_sources": [],
#     "model_sources": []
#   }

## DPO Fine Tuning Evaluation using LLM as a Judge (GPT-3.5 Turbo)
### LlaMA-3.2-1B-Instruct

In [1]:
!pip install -q transformers openai rouge_score nltk deepeval mysql.connector aiobotocore

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 10.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.8 MB/s eta 0:00:00
 

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
from openai import OpenAI
import json
import threading
from queue import Queue
import os
from huggingface_hub import login
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
import pandas as pd

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import json

with open('/kaggle/input/secrets/secrets.json') as f:
    secrets = json.load(f)

HF_TOKEN = secrets['HF_TOKEN']
OPEN_AI_KEY = secrets['OPEN_AI_KEY']
AWS_ACCESS_KEY_ID = secrets['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = secrets['AWS_SECRET_ACCESS_KEY']
DB_USER = secrets['DB_USER']
DB_PASSWORD = secrets['DB_PASSWORD']
DB_ENDPOINT = secrets['DB_ENDPOINT']
DB_PORT = secrets['DB_PORT']
DB_NAME = secrets['DB_NAME']

In [ ]:
api_key = OPEN_AI_KEY
hf_token = HF_TOKEN
login(token=hf_token)
os.environ["OPENAI_API_KEY"] = api_key

In [5]:
class LLMEvaluator:
    def __init__(self, judge_model: str = "gpt-3.5-turbo", api_key: str = None):
        if api_key is None:
            raise ValueError("API key must be provided either via parameter or OPENAI_API_KEY environment variable")
        self.judge_model = judge_model
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction()
        self.client = OpenAI(api_key=api_key)

    def calculate_geval_reasoning(self, question: str, reference: str, hypothesis: str) -> tuple[float, str]:
        reasoning_criteria = (
            "Evaluate the chain-of-thought reasoning in the generated text compared to the reference. "
            "Assess whether the reasoning steps are logical, complete, and lead to the correct conclusion."
        )
        
        evaluation_steps = [
            "Check if each reasoning step in the hypothesis is explicitly stated and follows logically from the previous step.",
            "Verify that all critical steps from the reference are present in the hypothesis.",
            "Determine if the final conclusion matches the reference and is supported by the reasoning."
        ]
        
        test_case = LLMTestCase(
            input="Solve this problem step-by-step: "+question,
            actual_output=hypothesis,
            expected_output=reference
            # evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
        )
        
        reasoning_metric = GEval(
            name="ReasoningQuality",
            criteria=reasoning_criteria,
            evaluation_steps=evaluation_steps,
            evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
            model=self.judge_model,
            threshold=0.7
            # api_key=self.api_key
        )
        
        reasoning_metric.measure(test_case)
        return reasoning_metric.score, reasoning_metric.reason

In [ ]:
class ModelGenerator:
    def __init__(self, baseline_path: str):
        # if torch.cuda.device_count() < 2:
        #     raise RuntimeError("Need at least 2 GPUs for this setup")
        
        self.device_baseline = torch.device("cuda")
        self.baseline_model = AutoModelForCausalLM.from_pretrained(baseline_path).to(self.device_baseline)
        self.baseline_tokenizer = AutoTokenizer.from_pretrained(baseline_path)

    def generate_response(self, question: str, model_type: str = "baseline") -> str:
        model = self.baseline_model
        tokenizer = self.baseline_tokenizer
        device = self.device_baseline
        
        prompt = f"Question: {question}\nPlease provide a step-by-step solution:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=1000, temperature=0.7, do_sample=True)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
def evaluate_responses(evaluator, question, ground_truth, baseline_response):

    baseline_geval_score, baseline_geval_reason = evaluator.calculate_geval_reasoning(question, ground_truth, baseline_response)
    
    baseline_metrics = {
        'geval_reasoning_score': baseline_geval_score,
        'geval_reasoning_reason': baseline_geval_reason
    }
    
    return {'baseline': baseline_metrics}

In [15]:
def generate_response_thread(generator, question, model_type, result_queue):
    response = generator.generate_response(question, model_type)
    result_queue.put((model_type, response))

def evaluate_multiple_questions(questions_ground_truths, generator, evaluator):
    results = {}
    
    for question, ground_truth in questions_ground_truths.items():
        result_queue = Queue()
        
        baseline_thread = threading.Thread(target=generate_response_thread, 
                                         args=(generator, question, "baseline", result_queue))
        
        baseline_thread.start()
        baseline_thread.join()
        
        responses = {}
        while not result_queue.empty():
            model_type, response = result_queue.get()
            responses[model_type] = response
        
        baseline_response = responses["baseline"]
        
        metrics = evaluate_responses(evaluator, question, ground_truth, baseline_response)
        
        results[question] = {
            'ground_truth': ground_truth,
            'baseline_response': baseline_response,
            'metrics': metrics
        }
        
        print(f"\nQuestion: {question}")
        # print("Baseline Response:\n", baseline_response)
        # print("Metrics:", metrics)
    
    return results

def results_to_dataframe(evaluation_results):
    # Prepare data for DataFrame
    data = []
    for question, result in evaluation_results.items():
        row = {'question': question, 'ground_truth': result['ground_truth'],
               'baseline_response': result['baseline_response']}
        
        # Add baseline metrics
        for metric, value in result['metrics']['baseline'].items():
            if metric == 'rouge':
                row.update({f'baseline_rouge1': value['rouge1'], f'baseline_rouge2': value['rouge2'], f'baseline_rougeL': value['rougeL']})
            else:
                row[f'baseline_{metric}'] = value
        
        data.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

In [9]:
def create_questions_dict():
    df = pd.read_csv("/kaggle/input/cot-llm-dpo/MATH_test_staging.csv")
    df_main = df.sample(20)
    df_main['ground_truth'] = df_main['reasoning']+' '+df_main['answer']
    df_main = df_main.drop(columns=['dataset_id', 'question_id','reasoning', 'answer'])
    df_main_dict = dict(zip(df_main['question_text'], df_main['ground_truth']))
    return df_main_dict

In [13]:
if __name__ == "__main__":
    OPENAI_API_KEY = api_key
    BASELINE_PATH = "Eshita-ds/Llama-3.2-1B-DPO"

    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    if torch.cuda.device_count() >= 2:
        print(f"GPU 0: {torch.cuda.get_device_name(0)}")
        print(f"GPU 1: {torch.cuda.get_device_name(1)}")

    generator = ModelGenerator(BASELINE_PATH)
    evaluator = LLMEvaluator("gpt-3.5-turbo", api_key=OPENAI_API_KEY)

    questions_ground_truths = create_questions_dict()

    evaluation_results = evaluate_multiple_questions(questions_ground_truths, generator, evaluator)

    with open('evaluation_results.json', 'w', encoding='utf-8') as f:
        json.dump(evaluation_results, f, indent=4, ensure_ascii=False)

    print("\nResults have been saved to 'evaluation_results.json'")

Number of GPUs available: 2
GPU 0: Tesla T4
GPU 1: Tesla T4


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Output()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Question: Let $f(x) = ax + b,$ where $a$ and $b$ are constants.  If $f(f(f(x))) = 8x + 21,$ find $a + b.$
Baseline Response:
 Question: Let $f(x) = ax + b,$ where $a$ and $b$ are constants.  If $f(f(f(x))) = 8x + 21,$ find $a + b.$
Please provide a step-by-step solution: 

## Step 1: Substitute $f(x)$ into $f(f(x))$ to find $f(f(f(x)))$
We have that \begin{align*}
f(f(x)) &= a(ax+b)+b\\
&= a^2x + ab + b.
\end{align*}

## Step 2: Substitute $f(f(x))$ into $f(f(f(x)))$ to find $f(f(f(x)))$
We have that \begin{align*}
f(f(f(x))) &= a(a^2x+ab+b)+b\\
&= a^3x + a^2b + ab + b.
\end{align*}

## Step 3: Equate the expression for $f(f(f(x)))$ to $8x+21$ and compare coefficients
We have that \begin{align*}
a^3x + a^2b + ab + b &= 8x+21\\
\Rightarrow\qquad a^3x + a^2b + ab + b - 8x - 21 &= 0\\
\Rightarrow\qquad a^3x + (a^2-8)x + (a^2b + ab + b - 21) &= 0.
\end{align*}

## Step 4: Equate coefficients of $x^3$, $x^2$, $x$, and constant terms
Since we have $a^3=1$, $a^2-8=0$, and $a^2b + ab + b - 21

...

Output()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Question: Compute $\tan 150^\circ$.
Baseline Response:
 Question: Compute $\tan 150^\circ$.
Please provide a step-by-step solution: 
## Step 1: Recall the definition of tangent in terms of sine and cosine
$\tan \theta = \frac{\sin \theta}{\cos \theta}$.

## Step 2: Determine the values of sine and cosine for 150 degrees
Using the unit circle or reference triangles, we find that $\sin 150^\circ = \frac{1}{2}$ and $\cos 150^\circ = -\frac{\sqrt{3}}{2}$.

## Step 3: Substitute the values into the tangent formula
$\tan 150^\circ = \frac{\sin 150^\circ}{\cos 150^\circ} = \frac{\frac{1}{2}}{-\frac{\sqrt{3}}{2}}$.

## Step 4: Simplify the expression
$\tan 150^\circ = \frac{\frac{1}{2}}{-\frac{\sqrt{3}}{2}} = -\frac{1}{\sqrt{3}}$.

## Step 5: Rationalize the denominator if necessary
We can rationalize the denominator by multiplying both the numerator and denominator by $\sqrt{3}$: $-\frac{1}{\sqrt{3}} \cdot \frac{\sqrt{3}}{\sqrt{3}} = -\frac{\sqrt{3}}{3}$.

The final answer is: $\boxed{-\frac

Output()


Question: Expand the product $(x-2)^2(x+2)^2$. What is the product of the nonzero coefficients of the resulting expression, including the constant term?
Baseline Response:
 Question: Expand the product $(x-2)^2(x+2)^2$. What is the product of the nonzero coefficients of the resulting expression, including the constant term?
Please provide a step-by-step solution: 

## Step 1: Expand the product using the distributive property
To expand the product $(x-2)^2(x+2)^2$, we first apply the distributive property (also known as the FOIL method for binomials) to each factor. This involves multiplying each term in the first parentheses by each term in the second parentheses.

## Step 2: Apply the FOIL method for the first factor $(x-2)^2$
Using the FOIL method, we get:
\[
(x-2)^2 = x^2 - 4x + 4
\]

## Step 3: Apply the FOIL method for the second factor $(x+2)^2$
Using the FOIL method, we get:
\[
(x+2)^2 = x^2 + 4x + 4
\]

## Step 4: Multiply the expanded expressions together
Now we multiply the

In [16]:
import pandas as pd
df = results_to_dataframe(evaluation_results)
print("\nEvaluation Results DataFrame:")
df.head()


Evaluation Results DataFrame:


,question,ground_truth,baseline_response,baseline_geval_reasoning_score,baseline_geval_reasoning_reason
0,"Let $f(x) = ax + b,$ where $a$ and $b$ are con...",We have that\n\begin{align*}\nf(f(f(x))) &= f(...,"Question: Let $f(x) = ax + b,$ where $a$ and $...",0.285645,The final answer provided is incorrect as it d...
1,Determine the number of ways to arrange the le...,First we count the arrangements if the two E's...,Question: Determine the number of ways to arra...,0.127844,The final conclusion does not match the refere...
2,Find the number of positive integers $n$ for w...,The graph has a hole at $x = 7$ as long as the...,Question: Find the number of positive integers...,0.167010,The actual output correctly analyzes the behav...
3,"Let $\tau = \frac{1 + \sqrt{5}}{2},$ the golde...",From the formula for an infinite geometric ser...,"Question: Let $\tau = \frac{1 + \sqrt{5}}{2},$...",0.779110,The text follows all the evaluation steps prov...
4,What is the remainder when $2000+2001+2002+200...,"Since $2000,2001,\ldots,2006$ are $7$ consecut...",Question: What is the remainder when $2000+200...,0.701875,The solution provided follows the steps outlin...


In [25]:
avg_geval = df["baseline_geval_reasoning_score"].mean()

In [ ]:
from huggingface_hub import list_repo_commits

model_id = "Eshita-ds/Llama-3.2-1B-DPO"
token = HF_TOKEN
commit=""

try:
    revisions = list_repo_commits(model_id, repo_type="model", token=token)
    if revisions:
        latest = revisions[0]
        commit = latest.commit_id[:8]
        print(f"✅ Latest Commit for {model_id}:")
        print(f"  Commit: {latest.commit_id[:8]} | Date: {latest.created_at}")
    else:
        print(f"⚠️ No commits found for {model_id}")
except Exception as e:
    print(f"❌ Error fetching latest commit: {e}")

print (commit)

✅ Latest Commit for Eshita-ds/Llama-3.2-1B-DPO:
  Commit: a378a528 | Date: 2025-05-08 07:56:46+00:00
a378a528


In [ ]:
import mysql.connector

db_user = DB_USER
db_password = DB_PASSWORD
db_endpoint = DB_ENDPOINT
db_port = DB_PORT
db_name = DB_NAME

DB_CONFIG = {
    "host": db_endpoint,
    "port": db_port,
    "user": db_user,
    "password": db_password,
    "database": db_name,
}

conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor(buffered=True)

cursor.execute("""UPDATE cot_models SET latest_commit = %s, latest_geval = %s 
                    WHERE model_name = %s""", 
               (commit, float(avg_geval), BASELINE_PATH))
conn.commit()
conn.close()